In [ ]:
import $ivy.`org.typelevel::cats-core:2.1.0`

// These are all the imports you need for everything here
import cats.implicits._
import cats.data.Reader

# The Reader Monad
`cats.data.Reader` allows us to sequence a bunch of operations that all depend on one input. A common use is dependency injection. If we have a bunch of operations that all depend on some configuration, we can chain them together using a `Reader` to produce one large operation that accepts the configuration as a parameter.

In [ ]:
// We can create a Reader from a function A => B using Reader.apply
case class Cat(name: String, favoriteFood: String)

val catNameReader: Reader[Cat, String] =
  Reader(cat => cat.name)

// We can extract the function using .run and call it as usual
val catName = catNameReader.run(Cat("Garfield", "lasagne"))

Pretty simple. So far `Reader` looks just like a wrapper around a function. What advantage do readers give us over raw functions?

The power of `Reader` comes from it's `map` and `flatMap`.

In [ ]:
// map extends the computation by passing its result through a function
val greetKittyReader: Reader[Cat, String] =
  catNameReader.map(name => s"Hello $name")

val greetKitty = greetKittyReader.run(Cat("Heathcliff", "junk food"))

// flatMap allows us to combine readers that depend on the same input type
val feedKittyReader: Reader[Cat, String] =
  Reader(cat => s"Have a nice bowl of ${cat.favoriteFood}")

val greetAndFeedReader: Reader[Cat, String] =
  for {
    greet <- greetKittyReader
    feed <- feedKittyReader
  } yield s"$greet. $feed."

val greetAndFeedGarfield   = greetAndFeedReader(Cat("Garfield", "lasagne"))
val greetAndFeedHeathcliff = greetAndFeedReader(Cat("Heathcliff", "junk food"))

## Exercise
Lets create a complete example of using `Reader` to build a program that accepts configuration as a parameter. The configuration will consist of two databases: a list of valid users and a list of their passwords:

In [ ]:
case class Db(
  usernames: Map[Int, String],
  passwords: Map[String, String]
)

// Lets create a type alias for the Reader that will make the rest of the code shorter
// type DbReader[A] = ???

// Now lets implement these methods
def findUsername(userId: Int): DbReader[Option[String]] =
  ???

def checkPassword(username: String, password: String): DbReader[Boolean] =
  ???

// Finally, lets use these methods to create a checkLogin
def checkLogin(userId: Int, password: String): DbReader[Boolean] =
  ???

// Now lets test it out
val users = Map(
  1 -> "dade",
  2 -> "kate",
  3 -> "margo"
)

val passwords = Map(
  "dade" -> "zerocool",
  "kate" -> "acidburn",
  "margo" -> "secret"
)

val db = Db(users, passwords)

val login1 = checkLogin(1, "zerocool").run(db)
val login2 = checkLogin(1, "crashoverride").run(db)
val login3 = checkLogin(4, "davinci").run(db)

In [ ]:
case class Db(
  usernames: Map[Int, String],
  passwords: Map[String, String]
)

// Lets create a type alias for the Reader that will make the rest of the code shorter
type DbReader[A] = Reader[Db, A]

// Now lets implement these methods
def findUsername(userId: Int): DbReader[Option[String]] =
  Reader(db => db.usernames.get(userId))

def checkPassword(username: String, password: String): DbReader[Boolean] =
  Reader(db => db.passwords.get(username).contains(password))

// Finally, lets use these methods to create a checkLogin
def checkLogin(userId: Int, password: String): DbReader[Boolean] =
  for {
    username <- findUsername(userId)
    passwordOk <- username.map { username =>
                    checkPassword(username, password)
                  }.getOrElse {
                    false.pure[DbReader]
                  }
  } yield passwordOk

// Now lets test it out
val users = Map(
  1 -> "dade",
  2 -> "kate",
  3 -> "margo"
)

val passwords = Map(
  "dade" -> "zerocool",
  "kate" -> "acidburn",
  "margo" -> "secret"
)

val db = Db(users, passwords)

val login1 = checkLogin(1, "zerocool").run(db)
val login2 = checkLogin(1, "crashoverride").run(db)
val login3 = checkLogin(4, "davinci").run(db)